In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import citypy module to determine city name from coordinates
from citipy import citipy
# import datetime
from datetime import datetime
import requests
from config import weather_api_key
import time
# Import linear regression dependency
from scipy.stats import linregress
import os

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list to hold city names
cities = []

# Identify the cities for each longitude and latitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If city is unique we will add it to the cities list
    if city not in cities:
        cities.append(city)
# Print city count to confirm
print(len(cities))


623


In [4]:
city_data = []
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging
print('Beginning data retrieval    ')
print('----------------------------')

# Create counters
record_count = 1
set_count = 1
counter = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
     # Run an API request for each of the cities.
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        city_precip = city_weather['weather'][0]['main']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        try:
            rain_3h = city_weather['rain']['3h']
            rain_3h_inches = rain_3h / 25.4
        except:
            rain_3h_inches = 0
            pass
        try:
            snow_3h = city_weather['snow']['3h']
            snow_3h_inches = snow_3h / 25.4
        except:
            snow_3h_inches = 0
            pass
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather Description": city_weather_description,
                          "City Precip": city_precip,
                          "Rain in Inches": rain_3h_inches,
                          "Snow in Inches": snow_3h_inches})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
    
    #city_data[counter].update({"Rain in Inches":rain_3h_inches})
    #counter +=1          
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

Beginning data retrieval    
----------------------------
Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | bethel
Processing Record 3 of Set 1 | raudeberg
Processing Record 4 of Set 1 | hilo
Processing Record 5 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 6 of Set 1 | lasa
Processing Record 7 of Set 1 | benguela
Processing Record 8 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 9 of Set 1 | upernavik
Processing Record 10 of Set 1 | port alfred
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | maniitsoq
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | huntsville
Processing Record 16 of Set 1 | qingyuan
Processing Record 17 of Set 1 | vaini
Processing Record 18 of Set 1 | villa bruzual
Processing Record 19 of Set 1 | puerto ayora
Processing Record 20 of Set 1 | cape town
Processing Record 21 of Set 1 | jabiru
City not found. Sk

In [16]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description,City Precip,Rain in Inches,Snow in Inches
0,Busselton,-33.65,115.33,57.99,67,94,11.52,AU,2020-04-01 23:00:27,overcast clouds,Clouds,0.0,0.0
1,Bethel,41.37,-73.41,53.01,29,1,16.11,US,2020-04-01 23:00:27,clear sky,Clear,0.0,0.0
2,Raudeberg,61.99,5.14,37.99,86,100,17.22,NO,2020-04-01 23:00:28,light rain,Rain,0.0,0.0
3,Hilo,19.73,-155.09,80.60,57,1,5.82,US,2020-04-01 23:00:28,clear sky,Clear,0.0,0.0
4,Lasa,46.62,10.70,32.00,79,5,1.61,IT,2020-04-01 23:00:28,clear sky,Clear,0.0,0.0


In [17]:
city_data_df.columns=['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed','Country','Date','Current Weather','Type of Weather', 'Rain Inches(last 3h)', 'Snow Inches(last 3h)']

In [18]:
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,Busselton,-33.65,115.33,57.99,67,94,11.52,AU,2020-04-01 23:00:27,overcast clouds,Clouds,0.0,0.0
1,Bethel,41.37,-73.41,53.01,29,1,16.11,US,2020-04-01 23:00:27,clear sky,Clear,0.0,0.0
2,Raudeberg,61.99,5.14,37.99,86,100,17.22,NO,2020-04-01 23:00:28,light rain,Rain,0.0,0.0
3,Hilo,19.73,-155.09,80.60,57,1,5.82,US,2020-04-01 23:00:28,clear sky,Clear,0.0,0.0
4,Lasa,46.62,10.70,32.00,79,5,1.61,IT,2020-04-01 23:00:28,clear sky,Clear,0.0,0.0


In [19]:
city_data_df = city_data_df[['City','Country','Date','Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness','Wind Speed', 'Current Weather','Type of Weather', 'Rain Inches(last 3h)','Snow Inches(last 3h)']]

In [20]:
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,Busselton,AU,2020-04-01 23:00:27,-33.65,115.33,57.99,67,94,11.52,overcast clouds,Clouds,0.0,0.0
1,Bethel,US,2020-04-01 23:00:27,41.37,-73.41,53.01,29,1,16.11,clear sky,Clear,0.0,0.0
2,Raudeberg,NO,2020-04-01 23:00:28,61.99,5.14,37.99,86,100,17.22,light rain,Rain,0.0,0.0
3,Hilo,US,2020-04-01 23:00:28,19.73,-155.09,80.60,57,1,5.82,clear sky,Clear,0.0,0.0
4,Lasa,IT,2020-04-01 23:00:28,46.62,10.70,32.00,79,5,1.61,clear sky,Clear,0.0,0.0


In [21]:
save_path = os.path.join('Resources','WeatherPy_challenge.csv')
city_data_df.to_csv(save_path)